In [34]:
import pandas as pd
import numpy as np
import yfinance as yf
import json
import pickle
from dateutil.relativedelta import relativedelta

In [4]:
df = pd.read_csv("sexy.csv")

In [7]:
df

,Ticker,Quarter,Depreciation,EPS in Rs,Expenses +,Interest,Net Profit +,Other Income +,Profit before tax,Tax %
0,AARTIIND.NS,2022-06-01,72.0,3.75,1326.0,45.0,136.0,0.0,167.0,19.0
1,AARTIIND.NS,2022-09-01,73.0,3.42,1418.0,44.0,124.0,0.0,150.0,17.0
2,AARTIIND.NS,2022-12-01,82.0,3.77,1379.0,47.0,137.0,0.0,160.0,15.0
3,AARTIIND.NS,2023-03-01,84.0,4.11,1403.0,33.0,149.0,0.0,136.0,-10.0
4,AARTIIND.NS,2023-06-01,89.0,1.93,1214.0,40.0,70.0,0.0,71.0,1.0
...,...,...,...,...,...,...,...,...,...,...
3206,ZYDUSLIFE.NS,2024-06-01,215.0,14.11,4124.0,32.0,1482.0,63.0,1900.0,23.0
3207,ZYDUSLIFE.NS,2024-09-01,234.0,9.06,3776.0,25.0,920.0,68.0,1271.0,29.0
3208,ZYDUSLIFE.NS,2024-12-01,229.0,10.17,3882.0,32.0,1026.0,57.0,1184.0,15.0
3209,ZYDUSLIFE.NS,2025-03-01,238.0,11.64,4402.0,77.0,1244.0,-139.0,1672.0,25.0


In [8]:
import pandas as pd
import yfinance as yf

def add_average_prices(df):
    """
    Fetches historical stock data and adds the quarterly average price to the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with 'Ticker' and 'Quarter' columns.

    Returns:
        pd.DataFrame: The DataFrame with the new 'Average Stock Price' column.
    """
    # --- Step 1: Prepare Tickers and Date Range ---
    # Ensure Quarter column is in datetime format for proper filtering
    df['Quarter'] = pd.to_datetime(df['Quarter'])

    # Get a list of all unique tickers and the overall date range needed
    tickers = df['Ticker'].unique().tolist()
    start_date = df['Quarter'].min()
    # Calculate the end date by adding 3 months to the last quarter's start date
    end_date = df['Quarter'].max() + pd.DateOffset(months=3)

    # --- Step 2: Download All Stock Data at Once ---
    # This is much more efficient than one call per row.
    # The downloaded data will have multi-level columns, e.g., ('Close', 'AARTIIND.NS')
    print("Downloading historical stock data for all tickers...")
    all_data = yf.download(tickers, start=start_date, end=end_date, progress=False)
    print("Download complete.")

    if all_data.empty:
        print("Could not download any stock data. Please check ticker symbols.")
        df['Average Stock Price'] = None
        return df

    # We only need the 'Close' prices for our calculation
    close_prices = all_data['Close']

    # --- Step 3: Calculate Averages for each row ---
    def calculate_avg_for_row(row):
        ticker = row['Ticker']
        quarter_start = row['Quarter']
        quarter_end = quarter_start + pd.DateOffset(months=3)

        # Check if we have data for this specific ticker
        if ticker not in close_prices.columns:
            return None

        # Select the 'Close' prices for the current row's ticker and quarter
        quarterly_prices = close_prices[ticker][(close_prices.index >= quarter_start) & (close_prices.index < quarter_end)]

        # Calculate the mean price, if any data exists for that period
        if not quarterly_prices.empty:
            return quarterly_prices.mean()
        else:
            return None

    print("Calculating quarterly averages...")
    df['Average Stock Price'] = df.apply(calculate_avg_for_row, axis=1)
    print("Calculation complete.")

    return df

In [9]:
# --- Main Execution ---
try:
    # Load your dataset
    df = pd.read_csv('sexy.csv')
    
    # Run the function to add the new column
    updated_df = add_average_prices(df)

    # Display the first few rows of the result
    print("\n--- Updated DataFrame ---")
    print(updated_df.head())

    # Save the final result to a new CSV file
    updated_df.to_csv('sexy_with_avg_price.csv', index=False)
    print("\nSuccessfully saved the updated data to 'sexy_with_avg_price.csv'")

except FileNotFoundError:
    print("Error: 'sexy.csv' not found. Please ensure the file is in the same directory as the script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

C:\Users\mjaye\AppData\Local\Temp\ipykernel_14348\1154276394.py:28: FutureWarning: YF.download() has changed argument auto_adjust default to True
  all_data = yf.download(tickers, start=start_date, end=end_date, progress=False)


Download complete.
Calculating quarterly averages...
Calculation complete.

--- Updated DataFrame ---
        Ticker    Quarter  Depreciation  EPS in Rs  Expenses +  Interest  \
0  AARTIIND.NS 2022-06-01          72.0       3.75      1326.0      45.0   
1  AARTIIND.NS 2022-09-01          73.0       3.42      1418.0      44.0   
2  AARTIIND.NS 2022-12-01          82.0       3.77      1379.0      47.0   
3  AARTIIND.NS 2023-03-01          84.0       4.11      1403.0      33.0   
4  AARTIIND.NS 2023-06-01          89.0       1.93      1214.0      40.0   

   Net Profit +  Other Income +  Profit before tax  Tax %  Average Stock Price  
0         136.0             0.0              167.0   19.0           741.980471  
1         124.0             0.0              150.0   17.0           753.102151  
2         137.0             0.0              160.0   15.0           580.257191  
3         149.0             0.0              136.0  -10.0           524.293072  
4          70.0             0.0     

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3211 entries, 0 to 3210
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ticker               3211 non-null   object        
 1   Quarter              3211 non-null   datetime64[ns]
 2   Depreciation         3211 non-null   float64       
 3   EPS in Rs            3211 non-null   float64       
 4   Expenses +           3211 non-null   float64       
 5   Interest             3211 non-null   float64       
 6   Net Profit +         3211 non-null   float64       
 7   Other Income +       3211 non-null   float64       
 8   Profit before tax    3211 non-null   float64       
 9   Tax %                3211 non-null   float64       
 10  Average Stock Price  3201 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 276.1+ KB


In [12]:
df.fillna(method='ffill', inplace=True)

C:\Users\mjaye\AppData\Local\Temp\ipykernel_14348\4002983762.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [21]:
df

,Ticker,Quarter,Depreciation,EPS in Rs,Expenses +,Interest,Net Profit +,Other Income +,Profit before tax,Tax %,Average Stock Price
0,AARTIIND.NS,2022-06-01,72.0,3.75,1326.0,45.0,136.0,0.0,167.0,19.0,741.980471
1,AARTIIND.NS,2022-09-01,73.0,3.42,1418.0,44.0,124.0,0.0,150.0,17.0,753.102151
2,AARTIIND.NS,2022-12-01,82.0,3.77,1379.0,47.0,137.0,0.0,160.0,15.0,580.257191
3,AARTIIND.NS,2023-03-01,84.0,4.11,1403.0,33.0,149.0,0.0,136.0,-10.0,524.293072
4,AARTIIND.NS,2023-06-01,89.0,1.93,1214.0,40.0,70.0,0.0,71.0,1.0,479.543615
...,...,...,...,...,...,...,...,...,...,...,...
3206,ZYDUSLIFE.NS,2024-06-01,215.0,14.11,4124.0,32.0,1482.0,63.0,1900.0,23.0,1133.486494
3207,ZYDUSLIFE.NS,2024-09-01,234.0,9.06,3776.0,25.0,920.0,68.0,1271.0,29.0,1019.727555
3208,ZYDUSLIFE.NS,2024-12-01,229.0,10.17,3882.0,32.0,1026.0,57.0,1184.0,15.0,950.947210
3209,ZYDUSLIFE.NS,2025-03-01,238.0,11.64,4402.0,77.0,1244.0,-139.0,1672.0,25.0,878.099550


In [16]:
df.columns

Index(['Ticker', 'Quarter', 'Depreciation', 'EPS in Rs', 'Expenses +',
       'Interest', 'Net Profit +', 'Other Income +', 'Profit before tax',
       'Tax %', 'Average Stock Price'],
      dtype='object')

In [32]:
len(df['Ticker'].unique())

247

In [25]:
TARGET_COLUMN = 'Average Stock Price'
CONTEXT_LENGTH = 4
PREDICTION_LENGTH = 2

df = pd.read_csv('sexy_with_avg_price.csv')
df = df.sort_values(by=['Ticker', 'Quarter'])

feature_columns = [
    col for col in df.select_dtypes(include=np.number).columns 
    if col != TARGET_COLUMN
]

print(f"Target Column: {TARGET_COLUMN}")
print(f"Feature Columns: {feature_columns}")

# Create a mapping for static categorical features (Ticker -> integer ID)
ticker_map = {ticker: i for i, ticker in enumerate(df['Ticker'].unique())}

# --- Convert Data into Time Series Format ---
training_data = []
testing_data = []

# Group by ticker to process each time series individually
for ticker, group in df.groupby('Ticker'):
    # Get the start date of the series
    start_date = group['Quarter'].min()
    
    # Extract target values
    target_values = group[TARGET_COLUMN].values
    
    # Extract dynamic features
    dynamic_features = group[feature_columns].values.T # Transpose to get (num_features, series_length)
    
    # The training target is the series minus the last few data points
    train_target = target_values[:-PREDICTION_LENGTH]
    
    training_data.append({
        'start': start_date,
        'target': train_target.tolist(),
        'feat_static_cat': [ticker_map[ticker]],
        'feat_dynamic_real': [feat[:-PREDICTION_LENGTH].tolist() for feat in dynamic_features]
    })

    # The test target includes the full series, as the model needs the beginning to predict the end
    testing_data.append({
        'start': start_date,
        'target': target_values.tolist(),
        'feat_static_cat': [ticker_map[ticker]],
        'feat_dynamic_real': [feat.tolist() for feat in dynamic_features]
    })


# You can now pass `training_data` and `testing_data` to a library like GluonTS
# from gluonts.dataset.common import ListDataset
# train_ds = ListDataset(training_data, freq="QS-DEC") # QS-DEC for quarterly start, fiscal year ending Dec
# test_ds = ListDataset(testing_data, freq="QS-DEC")



Target Column: Average Stock Price
Feature Columns: ['Depreciation', 'EPS in Rs', 'Expenses\xa0+', 'Interest', 'Net Profit\xa0+', 'Other Income\xa0+', 'Profit before tax', 'Tax %']


In [35]:
len(training_data)

247

In [36]:
len(testing_data)

247

In [26]:
print(f"\n training data is {len(training_data)} time series.")
print(f"Training data points per series: {len(training_data[0]['target'])}")
print(f"Testing data points per series: {len(testing_data[0]['target'])}")


 training data is 247 time series.
Training data points per series: 11
Testing data points per series: 13


In [27]:
print("\nExample of the first training series object:")
print(training_data[0])


Example of the first training series object:
{'start': '2022-06-01', 'target': [741.980470687624, 753.1021511939264, 580.2571914915054, 524.2930720011393, 479.54361486434937, 495.94290822347006, 627.5046524540071, 675.1451633259402, 679.4920230988533, 524.714855071037, 427.6349558830261], 'feat_static_cat': [0], 'feat_dynamic_real': [[72.0, 73.0, 82.0, 84.0, 89.0, 93.0, 97.0, 98.0, 102.0, 108.0, 111.0], [3.75, 3.42, 3.77, 4.11, 1.93, 2.51, 3.42, 3.64, 3.78, 1.43, 1.27], [1326.0, 1418.0, 1379.0, 1403.0, 1214.0, 1221.0, 1473.0, 1489.0, 1546.0, 1431.0, 1608.0], [45.0, 44.0, 47.0, 33.0, 40.0, 58.0, 54.0, 59.0, 64.0, 62.0, 85.0], [136.0, 124.0, 137.0, 149.0, 70.0, 91.0, 124.0, 132.0, 137.0, 52.0, 46.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, -1.0, 6.0, 7.0, 1.0], [167.0, 150.0, 160.0, 136.0, 71.0, 82.0, 116.0, 126.0, 145.0, 34.0, 40.0], [19.0, 17.0, 15.0, -10.0, 1.0, -11.0, -7.0, -5.0, 6.0, -53.0, -15.0]]}


In [39]:
json_filepath = 'training_data.json'
with open(json_filepath, 'w') as json_file:
    json.dump(training_data, json_file, indent=4)
print(f"Data successfully saved to {json_filepath}")

Data successfully saved to training_data.json


In [40]:
json_filepath = 'testing_data.json'
with open(json_filepath, 'w') as json_file:
    json.dump(testing_data, json_file, indent=4)
print(f"Data successfully saved to {json_filepath}")

Data successfully saved to testing_data.json


In [ ]:
# pickle_filepath = 'training_data.pkl'
# with open(pickle_filepath, 'wb') as f:
#     pickle.dump(training_samples, f)
# print(f"Data successfully saved to {pickle_filepath}")